---
jupyter: python3
toc: true
toc-depth: 3
number-sections: true
title: Pandas_07_데이터 집계
date: 2021-11-05 00:06
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

In [1]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display_html, display

In [2]:
def display_multiple_dfs(dfs:list, styles, margin=10):
    display_target = ''
    for each_df in dfs:
        each_df_html = each_df[0].style.set_caption(f'<b>{each_df[1]}</b>').set_table_styles(styles).set_table_attributes(f"style='display:inline;margin:{margin}px'")._repr_html_()
        display_target += each_df_html
    display_html(display_target, raw = True)

In [3]:
styles = [
    {"selector" : "caption", "props" : "text-align:center; font-size:16px"}
]

# groupby : agg
- {컬럼명 : 집계함수} 구조의 dictionary를 변수로 주면, 각 컬럼마다 할당된 집계함수로 집계한다.
    - 집계함수는 커스텀 함수를 사용해도 됨. 아래 코드처럼.

In [4]:
df = pd.DataFrame({
    'month': ['10월', '10월', '10월', '10월', '11월', '12월', '12월'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})

def mean_and_sqrt(x): # 이런 묘한 출력값이 필요한 상황이 있다고 치자.
        return np.sqrt(np.mean(x))
    
df.groupby(by=['month','fruits']).agg({'price':mean_and_sqrt, 'quantity':np.sum})

price  quantity
month fruits                     
10월   apple   10.000000         1
      banana  16.583124         7
      orange  14.142136         2
11월   apple   12.247449         5
12월   apple   14.142136         6
      banana  20.000000         7

- 데이터 집계 방식 중, 가장 범용성이 좋고, 함수화하기도 편한 방식이었다.

- 실제로 사용했던 예시

In [ ]:
agg_dict = {key:np.sum for key in target_column} # 전부 집계함수 sum으로 세팅
agg_dict['accountId'] = 'count' # accountId 수만 count로 세팅. accountId 수를 센다.

ready_for_stacked = for_stackbar.groupby(by=['log_cate']).agg(agg_dict) # 집계 적용

## by에 아무 컬럼도 할당 안 하고 싶으면 어떻게 해야 할까? (전체총합)

In [6]:
df.groupby(lambda x: 'total').agg({'quantity':'sum'})

,quantity
total,28


## 그룹 별로 랜덤 샘플 추출하기
`DataFrameGroupBy.sample(n=None, frac=None, replace=False, weights=None, random_state=None)[source]`
- n : int, optional. 각 그룹 별로 몇 개씩 뽑을지. frac과 동시에 사용할 수 없음. replace가 True가 아니라면, 가장 작은 그룹의 크기보다 클 수 없음.
- frac : float, optional. 각 그룹 별로 추출할 비율. 0 ~ 1 사이의 값을 넣는다.
- replace : bool, default False. 같은 행을 다시 뽑을 수 있는지. True or False
- weights : list-like, optional. 가중치. 기본값 None이면 확률 가중은 모두 같다. 대상 Dataframe의 길이와 같은 길이의 list-like 객체를 할당하면, 각 그룹을 정규화 한 후 샘플링 확률로 사용한다. 각 값들은 음수이면 안 되며, 각 그룹마다 최소 하나의 양수가 있어야 한다.
- random_state : int, array-like, BitGenerator, np.random.RandomState, np.random.Generator, optional. int, 배열형 또는 BitGenerator인 경우 난수 생성용 시드로 쓴다. np.random.RandomState 또는 np.random.Generator인 경우, 그대로 사용한다.

In [7]:
df_sample = pd.DataFrame({"a": ["red"] * 2 + ["blue"] * 2 + ["black"] * 2, "b": range(6)})
df_sample

,a,b
0,red,0
1,red,1
2,blue,2
3,blue,3
4,black,4
5,black,5


## 같은 비율로 추출

In [8]:
df_sample.groupby("a").sample(frac=0.5, random_state=42)

,a,b
5,black,5
2,blue,2
1,red,1


## 같은 수를 추출

In [9]:
df_sample.groupby("a").sample(n=1, random_state=1)

,a,b
4,black,4
2,blue,2
1,red,1


## 가중치 사용

In [10]:
df_sample.groupby("a").sample(
    n=1,
    weights=[1, 1, 1, 0, 0, 1],
    random_state=99,
)
# black과 blue에서는, 무조건 5와 2가 나오게 된다. 가중치를 그렇게 설정하였기 때문.

,a,b
5,black,5
2,blue,2
1,red,1
